### CR_1.13 - DesOracle

In [2]:
from pwn import *
from Crypto.Cipher import DES
from Crypto.Util.Padding import unpad
import binascii

# Impostazioni di connessione
HOST = 'desoracle.challs.cyberchallenge.it'
PORT = 9035
conn = remote(HOST, PORT)

def send_option(opt):
    conn.recvuntil(b'> ')
    conn.sendline(str(opt).encode())

def encrypt_text(text_hex, key_hex):
    send_option(1)  # Opzione: Encrypt text
    conn.recvuntil(b'in hex)? ')
    conn.sendline(text_hex.encode())
    conn.recvuntil(b'in hex)? ')
    conn.sendline(key_hex.encode())
    return conn.recvline().strip().decode()

###########################
# STEP 1: Recupera l'OTP #
###########################

# Inviamo 8 byte di zero con chiave = 8 byte zero
plaintext_hex = '00' * 8
key_hex = '00' * 8
ct_hex = encrypt_text(plaintext_hex, key_hex)
print("[*] Ciphertext ottenuto:", ct_hex)
ct = bytes.fromhex(ct_hex)
assert len(ct) % 8 == 0, "Ciphertext non multiplo di 8 byte!"

# Ci aspettiamo 16 byte: due blocchi da 8 byte
block1 = ct[:8]
block2 = ct[8:16]

# Prepariamo la cifratura DES con chiave zero
key0 = bytes.fromhex(key_hex)
cipher0 = DES.new(key0, DES.MODE_ECB)

# Il secondo blocco di padding, essendo il padding per un blocco da 8 byte,
# è costante: pad_block = 0x08 * 8.
pad_block = b'\x08' * 8
# Calcoliamo DES_0(pad_block)
B2 = cipher0.encrypt(pad_block)

# Dato che: block2 = B2 XOR OTP  =>  OTP = B2 XOR block2
OTP = bytes([B2[i] ^ block2[i] for i in range(8)])
print("[+] OTP recuperato:", OTP.hex())

####################################
# STEP 2: Recupera la flag cifrata #
####################################

# Ora scegliamo di usare la chiave 0 anche per cifrare la flag
send_option(2)  # Opzione: Encrypt flag
conn.recvuntil(b'in hex)? ')
conn.sendline(key_hex.encode())
flag_ct_hex = conn.recvline().strip().decode()
print("[*] Flag ciphertext:", flag_ct_hex)
flag_ct = bytes.fromhex(flag_ct_hex)

# La flag è stata cifrata come:
# flag_ct = DES_0(pad(flag XOR OTP)) XOR OTP
# Per decifrare, per ogni blocco di 8 byte:
#   1. XOR: block_dec = blocco XOR OTP
#   2. Decripta con DES a chiave zero.
#   3. Dopo aver ricostruito il messaggio con padding, rimuovi il padding
#   4. Infine, ricava flag = (ricostruito) XOR OTP

plaintext_padded = b""
# Il ciphertext potrebbe essere composto da N blocchi da 8 byte.
for i in range(0, len(flag_ct), 8):
    block = flag_ct[i:i+8]
    # Rimuovo l'OTP: per ogni byte del blocco
    block_xor = bytes([block[j] ^ OTP[j] for j in range(8)])
    # Decripto il blocco con DES a chiave zero
    plaintext_padded += cipher0.decrypt(block_xor)

# Rimuovo il padding PKCS#7
try:
    data = unpad(plaintext_padded, 8)
except Exception as e:
    print("Errore nell'unpadding:", e)
    data = plaintext_padded

# Ora, ricordiamo che il messaggio de-padding è: flag XOR OTP.
# Quindi, per ottenere la flag:
flag_bytes = bytes([data[i] ^ OTP[i % 8] for i in range(len(data))])
flag = flag_bytes.decode()
print("[+] Flag trovata:")
print(flag)

conn.close()


[x] Opening connection to desoracle.challs.cyberchallenge.it on port 9035
[x] Opening connection to desoracle.challs.cyberchallenge.it on port 9035: Trying 5.75.221.48
[+] Opening connection to desoracle.challs.cyberchallenge.it on port 9035: Done
[*] Ciphertext ottenuto: f8057bc5856258c070bfb2f53ea7563d
[+] OTP recuperato: 0efd9ad7499130fd
[*] Flag ciphertext: 1c3e78e283991f5276dc43528d550acf351fb8a3db853e2259e095606f2438a9
[+] Flag trovata:
CCIT{50m3_k3y5_c4n_b3_w34k}
[*] Closed connection to desoracle.challs.cyberchallenge.it port 9035
